In [1]:
import os, random, numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [2]:
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

In [3]:
# -------------------- 2. Load Dataset --------------------
(num_classes, base_input_shape) = (10, (32,32,3))
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = y_train.flatten()
y_test  = y_test.flatten()

# Train/Val Split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=SEED, stratify=y_train
)

# Ensure RGB (3 channels)
if x_train.shape[-1] == 1:  # sometimes grayscale by accident
    x_train = np.repeat(x_train, 3, axis=-1)
    x_val   = np.repeat(x_val, 3, axis=-1)
    x_test  = np.repeat(x_test, 3, axis=-1)


In [4]:
IMG_SIZE = 224
preprocess = keras.applications.efficientnet.preprocess_input

In [5]:
def make_ds(x, y, train=False, batch_size=64):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    if train:
        ds = ds.shuffle(10000, seed=SEED)
    # Resize and preprocess
    ds = ds.map(lambda a, b: (tf.image.resize(a, (IMG_SIZE, IMG_SIZE)), b),
                num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(lambda a, b: (preprocess(a), b), num_parallel_calls=tf.data.AUTOTUNE)
    if train:
        # Light augmentation
        aug = keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.05),
            layers.RandomZoom(0.1),
        ])
        ds = ds.map(lambda a, b: (aug(a, training=True), b),
                    num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [6]:
train_ds = make_ds(x_train, y_train, train=True, batch_size=128)
val_ds   = make_ds(x_val,   y_val,   train=False, batch_size=128)
test_ds  = make_ds(x_test,  y_test,  train=False, batch_size=128)

In [7]:
# Force 3 channels in the model definition
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base = keras.applications.EfficientNetB0(
    include_top=False, weights=None, input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base.trainable = False


In [8]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model_tl = keras.Model(inputs, outputs, name="efficientnetb0_cifar10")

model_tl.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model_tl.summary()

Model: "efficientnetb0_cifar10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,062,381 (15.50 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [9]:
# -------------------- 5. Warmup Training --------------------
print("\n[Stage 1] Training head only...")
warmup = model_tl.fit(
    train_ds, validation_data=val_ds,
    epochs=5, verbose=2
)


[Stage 1] Training head only...
Epoch 1/5
352/352 - 1075s - 3s/step - accuracy: 0.0982 - loss: 2.3026 - val_accuracy: 0.1206 - val_loss: 2.3024
Epoch 2/5
352/352 - 912s - 3s/step - accuracy: 0.1088 - loss: 2.3024 - val_accuracy: 0.1240 - val_loss: 2.3022
Epoch 3/5
352/352 - 896s - 3s/step - accuracy: 0.1160 - loss: 2.3022 - val_accuracy: 0.1454 - val_loss: 2.3020
Epoch 4/5
352/352 - 621s - 2s/step - accuracy: 0.1268 - loss: 2.3020 - val_accuracy: 0.1476 - val_loss: 2.3018
Epoch 5/5
352/352 - 582s - 2s/step - accuracy: 0.1278 - loss: 2.3018 - val_accuracy: 0.1542 - val_loss: 2.3016


In [10]:
# -------------------- 6. Fine-Tuning --------------------
print("\n[Stage 2] Fine-tuning last 50 layers...")
for layer in base.layers[-50:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

model_tl.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


[Stage 2] Fine-tuning last 50 layers...


In [11]:
early = keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True, monitor="val_accuracy"
)

fine = model_tl.fit(
    train_ds, validation_data=val_ds,
    epochs=20, callbacks=[early], verbose=2
)

Epoch 1/20
352/352 - 697s - 2s/step - accuracy: 0.1394 - loss: 2.2218 - val_accuracy: 0.1820 - val_loss: 2.1128
Epoch 2/20
352/352 - 746s - 2s/step - accuracy: 0.1949 - loss: 2.1244 - val_accuracy: 0.2446 - val_loss: 2.0558
Epoch 3/20
352/352 - 690s - 2s/step - accuracy: 0.2250 - loss: 2.0850 - val_accuracy: 0.2450 - val_loss: 2.0383
Epoch 4/20
352/352 - 737s - 2s/step - accuracy: 0.2324 - loss: 2.0632 - val_accuracy: 0.2466 - val_loss: 2.0189
Epoch 5/20
352/352 - 693s - 2s/step - accuracy: 0.2379 - loss: 2.0505 - val_accuracy: 0.2528 - val_loss: 2.0049
Epoch 6/20
352/352 - 700s - 2s/step - accuracy: 0.2416 - loss: 2.0407 - val_accuracy: 0.2554 - val_loss: 1.9960
Epoch 7/20
352/352 - 701s - 2s/step - accuracy: 0.2417 - loss: 2.0326 - val_accuracy: 0.2572 - val_loss: 1.9978
Epoch 8/20
352/352 - 699s - 2s/step - accuracy: 0.2468 - loss: 2.0259 - val_accuracy: 0.2544 - val_loss: 1.9988
Epoch 9/20
352/352 - 732s - 2s/step - accuracy: 0.2469 - loss: 2.0191 - val_accuracy: 0.2658 - val_loss:

In [12]:
# -------------------- 7. Evaluation --------------------
print("\n[Stage 3] Evaluating on test set...")
probs = model_tl.predict(test_ds, verbose=0)
y_pred = probs.argmax(axis=1)


[Stage 3] Evaluating on test set...


In [13]:
# Align true labels with test_ds order
y_true_list = []
for _, yb in test_ds:
    y_true_list.append(yb.numpy())
y_true = np.concatenate(y_true_list, axis=0)

acc = (y_pred == y_true).mean()
print(f"[EfficientNetB0 TL] Test Accuracy: {acc:.4f}")

print("\nClassification report:")
print(classification_report(y_true, y_pred, digits=4))

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)

[EfficientNetB0 TL] Test Accuracy: 0.2899

Classification report:
              precision    recall  f1-score   support

           0     0.3485    0.4140    0.3784      1000
           1     0.3279    0.3400    0.3338      1000
           2     0.1552    0.0430    0.0673      1000
           3     0.1268    0.0090    0.0168      1000
           4     0.2176    0.2690    0.2406      1000
           5     0.2628    0.3740    0.3087      1000
           6     0.2784    0.5900    0.3783      1000
           7     0.2350    0.2390    0.2370      1000
           8     0.3432    0.2660    0.2997      1000
           9     0.4142    0.3550    0.3823      1000

    accuracy                         0.2899     10000
   macro avg     0.2710    0.2899    0.2643     10000
weighted avg     0.2710    0.2899    0.2643     10000

Confusion matrix:
 [[414  74  11  10  25 116  39  86 178  47]
 [ 74 340   8   6  20  44  82  81 103 242]
 [109  37  43  10 213 189 276  72  38  13]
 [ 57  60  54   9 173 264 2

In [14]:
# Precision, Recall, F1
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"Macro Precision: {prec:.4f} | Macro Recall: {rec:.4f} | Macro F1: {f1:.4f}")
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='micro')
print(f"Micro Precision: {prec:.4f} | Micro Recall: {rec:.4f} | Micro F1: {f1:.4f}")

Macro Precision: 0.2710 | Macro Recall: 0.2899 | Macro F1: 0.2643
Micro Precision: 0.2899 | Micro Recall: 0.2899 | Micro F1: 0.2899


In [15]:
# ROC-AUC
y_true_oh = keras.utils.to_categorical(y_true, num_classes)
try:
    auc_ovr = roc_auc_score(y_true_oh, probs, average="macro", multi_class="ovr")
    auc_ovo = roc_auc_score(y_true_oh, probs, average="macro", multi_class="ovo")
    print(f"ROC-AUC (macro, OVR): {auc_ovr:.4f} | ROC-AUC (macro, OVO): {auc_ovo:.4f}")
except Exception as e:
    print("ROC-AUC could not be computed:", e)

ROC-AUC (macro, OVR): 0.7625 | ROC-AUC (macro, OVO): 0.7625
